In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337) # set a random seed number or not

In [ ]:
flagD = 1
th = 1
ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)

# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])

kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
kRecs0 = pd.read_csv('Coop_params.txt',sep="\t") # The list of parameters, which should be set equal to zero, list obtained from the Bouhaddou2018 model
kRecs = kRecs0.values[:,0]
# kRecs

In [ ]:
# prepare gene states
genedata, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)

In [ ]:
startTime = datetime.now()
Vn = 1.75E-12 
Vc = 5.25E-12

ligInds = [155,156,157,158,159,160,161] # E, H, HGF, P, F, I, INS
ligConc = [0.001,0.01,0.1,1,10,100,1000]
ResInds = [162,166,167,170,171,172,173,174] # E1, E3, E4, Met, Pr, Fr, Ir, Isr
RecConc = 60.8754593

SPInds = np.arange(188,194)
SPPInds = np.arange(194,214)
SPSPInds = np.arange(214,225)

conds = np.zeros(shape=(len(ligInds),len(ligConc)))
# set everything off except lig-rec stuff
for ii,kk in enumerate(kRecs): # see above for kRecs
    model.setFixedParameterById(kk,0.0)

# # set starting lig-rec species to zero - Non-zero in MCF10A context plots
# RLsInds = np.arange(162,188)
# spdata[RLsInds] = 0.0

for count,lig in enumerate(ligInds):
    for cnc in range(len(ligConc)):
#         spdata[ResInds] = RecConc # Comment-out for 10A context
        spdata2 = spdata.copy()
        spdata2[lig] = ligConc[cnc]    
        for qq in range(NSteps):
            genedata,AllGenesVec = gm(flagD,ts,genedata,spdata2,Vn,Vc,kGin,kGac, \
                                       kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
            for ii,kk in enumerate(kmRNAs):
                model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
            model.setInitialStates(spdata2)
            rdata = amici.runAmiciSimulation(model, solver)  # Run simulation
            spdata2 = rdata['x'][-1,:]
        ns_prod_sum = sum(rdata['x'][-1,SPInds]) + sum(rdata['x'][-1,SPPInds]) + 2.0*sum(rdata['x'][-1,SPSPInds])
        conds[count,cnc] = ns_prod_sum
    print(datetime.now() - startTime)
print(datetime.now() - startTime)

In [ ]:
condsDF = pd.DataFrame(data=conds,columns=[ele for ele in ligConc],index=[ele for ele in ligInds]) 
condsDF.to_csv('cooperativity_reproduce.txt',sep="\t")   

In [ ]:
# script from https://stackoverflow.com/questions/55078451/how-to-use-curvefit-in-python (accessed on Jan16, 2020)
from scipy.optimize import curve_fit
import warnings

# these are the same as the scipy defaults
initialParameters = np.array([1.0, 1.0, 1.0])

# do not print unnecessary warnings during curve_fit()
warnings.filterwarnings("ignore")

def func(x, a, b, c): # Hill sigmoidal equation
    return  a * np.power(x, b) / (np.power(c, b) + np.power(x, b)) 

Hillns = []
for count,lig in enumerate(ligInds):
    # curve fit the test data
    fittedParameters, pcov = curve_fit(func, ligConc, conds[count,:], initialParameters)
    Hillns.append(fittedParameters[1])
    modelPredictions = func(ligConc, *fittedParameters) 
    absError = modelPredictions - conds[count,:]
    SE = np.square(absError) # squared errors
    MSE = np.mean(SE) # mean squared errors
    RMSE = np.sqrt(MSE) # Root Mean Squared Error, RMSE
    Rsquared = 1.0 - (np.var(absError) / np.var(conds[count,:]))

    print('Parameters:', fittedParameters)
    print('RMSE:', RMSE)
    print('R-squared:', Rsquared)
    print()
print(Hillns)